### 격자번호 부여
##### notebook_pre 이후

In [ ]:
# 필요한 라이브러리 불러오기
import re
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from shapely.geometry import Point


In [ ]:
# 서울/경기/인천의 그리드정보 불러오기
# index 별 (=그리드 별) 위치정보만 취합
grid = pd.read_csv('check2_서인경_그리드정보.csv').reset_index()
grid = grid[['index', 'geometry']]

In [ ]:
# 정규표현식을 활용한 좌표 추출
p = re.compile('[0-9]+[.]?[0-9]+')
def coord_ex(strg):
  return list(map(float, p.findall(strg)))[::-1][:-2]

# grid의 'coord'에 좌표 부여
# coords에 각 그리드의 꼭짓점을 부여 (3차원)
# centers에 각 그리드의 무게중심점을 부여 (2차원)
grid['coord'] = grid['geometry'].apply(lambda x : coord_ex(x))
coords = np.array(grid['coord'].values.tolist()).reshape(-1, 4, 2)
centers = coords.mean(axis=1)

# 그리드 번호와 무게중심점을 저장
geo1 = pd.DataFrame(data = centers)
geo1.to_csv('check3_1_그리드_중심정보.csv')

# 그리드 번호와 꼭짓점을 저장
geo2 = pd.DataFrame(coords.reshape(-1, 8))
geo2.to_csv('check3_2_그리드_좌표정보.csv')

In [ ]:
# 운행데이터 불러오기
drive = pd.read_csv('check1_운행.csv', index_col = 0)

# 최근접점을 찾아주는 KDTree 활용
check_in = s_merge['geometry']
tree = KDTree(centers)

for idx, srs in 운행.iterrows():
  # dd : 최근접점까지의 거리 (Euc)  /  ii : 최근접점의 인덱스
  dd, ii = tree.query([srs['s_lat'], srs['s_lng']], k=1)
  drive.at[idx, 'grid'] = ii

  # 데이터가 최근접점(무게중심)의 실제 그리드 내에 있는지 판단
  drive.at[idx, 'in_TF'] = Point([srs['s_lng'], srs['s_lat']]).within(check_in[ii])

In [ ]:
# 격자범위 내에 없는 데이터는 -1번으로 부여
for idx, srs in drive.iterrows():
  if srs['in_TF']:
    continue
  else:
    drive.at[idx, 'grid'] = -1

# rough한 운행데이터 저장
drive.to_csv('check4_1_운행_모든속성.csv')
# 운행데이터 중 위치좌표/그리드만 추출
drive_loc = drive[['s_lat', 's_lng', 'e_lat', 'e_lng', 'grid']]
drive_loc.to_csv('check4_2_운행_좌표그리드.csv')

In [ ]:
# 외부데이터 불러오기 : 서울 지하철역 위도/경도
# 좌표 결측치 제거
subway = pd.read_csv('서울 지하철역 좌표.csv')
subway = subway[(subway['lat'].notnull()) & (subway['lng'].notnull())]

bus = pd.read_csv('전국버스정류장 위치정보.csv', encoding='cp949')
bus = bus[(bus['위도'].notnull()) & (bus['경도'].notnull())]
# 버스는 전국데이터이므로, 수도권만 추려내기
bus = bus[bus['정류장아이디'].str.contains('ICB|SEB|GGB')].rename(columns = {'위도' : 'lat', '경도' : 'lng'})

# 대중교통 합체
trans = pd.concat([subway[['lat', 'lng']], bus[['lat', 'lng']]], axis=0)

# 최근접점을 찾아주는 KDTree 활용
for idx, srs in trans.iterrows():
  # dd : 최근접점까지의 거리 (Euc)  /  ii : 최근접점의 인덱스
  dd, ii = tree.query([srs['lat'], srs['lng']], k=1)
  trans.at[idx, 'grid'] = ii

  # 데이터가 최근접점(무게중심)의 실제 그리드 내에 있는지 판단
  trans.at[idx, 'in_TF'] = Point([srs['lng'], srs['lat']]).within(check_in[ii])

In [ ]:
# 격자범위 내에 없는 데이터는 -1번으로 부여
for idx, srs in trans.iterrows():
  if srs['in_TF']:
    continue
  else:
    trans.at[idx, 'grid'] = -1
  
# 운행데이터 중 위치좌표/그리드만 추출
trans = trans[['lat', 'lng', 'grid']]
trans.to_csv('check4_3_대중교통_좌표그리드.csv')

In [ ]:
# 학습용 데이터 생성
# 그리드 정보 가져오기
grid_sik = pd.read_csv('/content/check2_서인경_그리드정보.csv', index_col = 0)
data = grid_sik[['고속도로', '고속화도로', '공연문화시설', '노후주택수', '병원', '생활권공원', '전기차충전소', '주차장']]

data.to_csv('check5_독립변수_전처리전.csv')